In [1]:
import jieba
import pandas as pd
import sklearn
import numpy as np

In [2]:
jieba.load_userdict("../data/dict")
import re

def strQ2B(ustring):
    ss = []
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss.append(rstring)
    return "".join(ss)

def change_sentence(sentence):
    # 全角转半角
    sentence = strQ2B(sentence)
    
    # 去除标点符号
    sentence = sentence.replace(",", "")
    sentence = sentence.replace("，", "")
    sentence = sentence.replace(".", "")
    sentence = sentence.replace("。", "")
    sentence = sentence.replace("?", "")
    sentence = sentence.replace("？", "")
    sentence = sentence.replace("!", "")
    
    # 替换某些词语
    sentence = sentence.replace("借贝", "借呗")
    sentence = sentence.replace("花贝", "花呗")
    sentence = sentence.replace("蚂蚁借呗", "借呗")
    sentence = sentence.replace("蚂蚁花呗", "花呗")
    sentence = sentence.replace("蚂蚁花呗", "花呗")
    sentence = sentence.replace("整么", "怎么")
    sentence = sentence.replace("冻解", "冻结")
    sentence = sentence.replace("撤掉", "撤销")
    sentence = sentence.replace("提额", "提高额度")
    sentence = sentence.replace("买机票", "订机票")
    
    # 将***替换成N
    sentence = re.sub(r'[*]+', "N", sentence)
    
    return sentence

def seg_dataset(dataset_path, save_path):
    """
    """
    dataset = pd.read_csv(dataset_path, sep="\t", header=None)
    
    dataset.columns = ["label", "sent1", "sent2", "id"]
    
    print(dataset.head(2))
    
    dataset["sent1"] = dataset["sent1"].apply(lambda x: "".join(str(x).split()))
    dataset["sent1"] = dataset["sent1"].apply(lambda x: change_sentence(x))
    
    #dataset["sent1"] = dataset["sent1"].apply(lambda x: " ".join(jieba.cut(x, HMM=False)))
    dataset["sent1"] = dataset["sent1"].apply(lambda x: " ".join(jieba.cut(x, HMM=True)))
    
    dataset["sent2"] = dataset["sent2"].apply(lambda x: str(x).replace(" ", ""))
    dataset["sent2"] = dataset["sent2"].apply(lambda x: change_sentence(x))
    #dataset["sent2"] = dataset["sent2"].apply(lambda x: " ".join(jieba.cut(x, HMM=False)))
    dataset["sent2"] = dataset["sent2"].apply(lambda x: " ".join(jieba.cut(x, HMM=True)))
    
    dataset.to_csv(save_path, sep="\t", header=False, index=False)
    
    return True

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.764 seconds.
Prefix dict has been built succesfully.


In [3]:
seg_dataset("../data/ori_data/train.csv", "../data/ori_data/train_process_hmm.csv")
seg_dataset("../data/ori_data/dev.csv", "../data/ori_data/dev_process_hmm.csv")

   label                   sent1          sent2      id
0      0  为什么 我 的 花呗 额度 总是 无法 提高     都 是 用 花呗 的  100209
1      0         花呗 冻结 后要 怎样才能 用  花呗 冻结 了 怎样 还款   57509
   label                                            sent1  \
0      0                      支付宝 在 不同 手机 登录 会 影响 借呗 使用 吗   
1      0  花呗 逾期 一个月 刚用 不 知道 这个 月 号 还 的 要 到 什么 时候 才能 用 很 急   

                           sent2     id  
0            网上 贷款 会 不会 影响 借呗 信用  54642  
1  我 现在 刚借 的 花呗 要 下个月 最迟 什么 时候 还  50309  


True

In [4]:
seg_dataset("../data/ori_data/complete.csv", "../data/ori_data/complete_process_hmm.csv")

   label                   sent1          sent2      id
0      0  为什么 我 的 花呗 额度 总是 无法 提高     都 是 用 花呗 的  100209
1      0         花呗 冻结 后要 怎样才能 用  花呗 冻结 了 怎样 还款   57509


True

In [5]:
# make the dev dataset into origin
def transform2origin(dataset_path, origin_dev_path):
    """
    :param dataset_path
    """
    dataset = pd.read_csv(dataset_path, sep="\t", header=None)
    
    dataset.columns = ["label", "sent1", "sent2", "id"]
    
    new_dataset = pd.DataFrame()
    new_dataset["id"] = dataset["id"]
    new_dataset["sent1"] = dataset["sent1"].apply(lambda x: "".join(x.split()))
    new_dataset["sent2"] = dataset["sent2"].apply(lambda x: "".join(x.split()))
    
    new_dataset.to_csv(origin_dev_path, sep="\t", header=False, index=False)
    
    return dataset["label"].values

In [21]:
label = transform2origin("../data/ori_data/dev.csv", "../data/ori_data/origin.dev.csv")

In [41]:
preds = []
with open("/home/xueyunzhe/examplex_learning/matching/SentencePairs/submit/vec_feat_xgb_test/test_pred", "r") as reader:
    for line in reader:
        lineno, label_ = line.strip().split("\t")
        preds.append(int(label_))
preds = np.array(preds)

In [42]:
preds

array([0, 0, 1, ..., 0, 0, 1])

In [43]:
from sklearn.metrics import classification_report
print(classification_report(label, preds))

             precision    recall  f1-score   support

          0       0.90      0.83      0.86     16751
          1       0.43      0.57      0.49      3744

avg / total       0.81      0.78      0.79     20495

